In [14]:
import pandas as pd

In [15]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder

In [16]:
df = pd.read_csv('covid_toy.csv')
df

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No
...,...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore,No
96,51,Female,101.0,Strong,Kolkata,Yes
97,20,Female,101.0,Mild,Bangalore,No
98,5,Female,98.0,Strong,Mumbai,No


In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['has_covid']), df['has_covid'], test_size=0.2)

In [18]:
df.isna().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

Without Using The Column Transformer

In [19]:
# adding simple imputer to fever col
si = SimpleImputer()
df['fever'] = si.fit_transform(df[['fever']]) #assigning mean to the missing value


In [20]:
df.isna().sum()

age          0
gender       0
fever        0
cough        0
city         0
has_covid    0
dtype: int64

In [21]:
# OrdinalEncoding -> cough
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])
df['cough'] = oe.fit_transform(df[['cough']])
df

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,0.0,Kolkata,No
1,27,Male,100.0,0.0,Delhi,Yes
2,42,Male,101.0,0.0,Delhi,No
3,31,Female,98.0,0.0,Kolkata,No
4,65,Female,101.0,0.0,Mumbai,No
...,...,...,...,...,...,...
95,12,Female,104.0,0.0,Bangalore,No
96,51,Female,101.0,1.0,Kolkata,Yes
97,20,Female,101.0,0.0,Bangalore,No
98,5,Female,98.0,1.0,Mumbai,No


In [22]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse_output=False).set_output(transform='pandas')
transformed = ohe.fit_transform(df[['gender', 'city']])
transformed.shape

(100, 4)

In [23]:
df = pd.concat([df,transformed], axis='columns')

In [24]:
df

,age,gender,fever,cough,city,has_covid,gender_Male,city_Delhi,city_Kolkata,city_Mumbai
0,60,Male,103.0,0.0,Kolkata,No,1.0,0.0,1.0,0.0
1,27,Male,100.0,0.0,Delhi,Yes,1.0,1.0,0.0,0.0
2,42,Male,101.0,0.0,Delhi,No,1.0,1.0,0.0,0.0
3,31,Female,98.0,0.0,Kolkata,No,0.0,0.0,1.0,0.0
4,65,Female,101.0,0.0,Mumbai,No,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
95,12,Female,104.0,0.0,Bangalore,No,0.0,0.0,0.0,0.0
96,51,Female,101.0,1.0,Kolkata,Yes,0.0,0.0,1.0,0.0
97,20,Female,101.0,0.0,Bangalore,No,0.0,0.0,0.0,0.0
98,5,Female,98.0,1.0,Mumbai,No,0.0,0.0,0.0,1.0


Now with using the Pipelines

In [25]:
from sklearn.compose import ColumnTransformer

In [28]:
transfer = ColumnTransformer(transformers = [
    
    ('tnf1', SimpleImputer(),[2]),
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), [3]),
    ('tnf3', OneHotEncoder(sparse_output=False,drop='first'), [2,4])
], remainder='passthrough')

In [36]:
x = transfer.fit_transform(X_train)